In [2]:
!pip install pandas tabula-py openpyxl
!pip install pdfplumber
!pip install pytesseract pillow pandas openpyxl
!pip install paddlepaddle paddleocr
!sudo apt install tesseract-ocr

import re
from paddleocr import PaddleOCR
import pdfplumber
import numpy as np
from openpyxl import Workbook

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF using PaddleOCR.
    """
    extracted_text = ""

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Convert the page to an image
            img = page.to_image()
            img_pil = img.original  # Get the PIL Image object

            # Convert the PIL Image to a NumPy array
            img_np = np.array(img_pil)

            # Pass the NumPy array to PaddleOCR for text extraction
            ocr_result = ocr.ocr(img_np, cls=True)
            for result in ocr_result[0]:
                extracted_text += result[1][0] + '\n'

    return extracted_text

def process_extracted_text_to_excel(text, output_file='output.xlsx'):
    """
    Process extracted text and save it into an Excel file with specified fields.
    """
    # Define the fields (column headers)
    fields = ["Lot no.Associated lots", "Unit no.", "U/E Lot no.Associated lots", "Unit no.", "U/E"]

    # Prepare data storage
    rows = []
    lines = text.split("\n")

    # Locate the start of data after "U/E"
    data_start_index = 0
    for i, line in enumerate(lines):
        if "U/E" in line:  # Look for the last "U/E" header
            data_start_index = i + 1

    # Process data starting after the last "U/E" header
    data_lines = lines[data_start_index:]
    cleaned_lines = [line.strip() for line in data_lines if line.strip()]  # Remove empty lines

    # Chunk data into groups of 5 to match the fields
    for i in range(0, len(cleaned_lines), 5):
        chunk = cleaned_lines[i:i + 5]
        if len(chunk) == 5:  # Ensure full rows
            rows.append(chunk)

    # Write data to Excel
    workbook = Workbook()
    sheet = workbook.active

    # Write headers
    sheet.append(fields)

    # Write rows
    for row in rows:
        sheet.append(row)

    # Save the Excel file
    workbook.save(output_file)
    print(f"Data has been saved to {output_file}")

def main(pdf_path):
    # Step 1: Extract text from the PDF
    extracted_text = extract_text_from_pdf(pdf_path)

    # Step 2: Process the text and save it to Excel
    process_extracted_text_to_excel(extracted_text)

# Main execution
pdf_path = '/content/test.pdf'  # Replace with your PDF file path
main(pdf_path)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
[2025/02/02 11:27:37] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=